In [ ]:
#Library thing scraper

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options 
from bs4 import BeautifulSoup, NavigableString
import pandas as pd
import numpy as np
import requests
import re

In [2]:
#this is the function that searches for the movie title and date
#it returns the title it finds (in case it is different than the search term) and a link to the first result
def MovieFirst(baseurl, title):
    
    #initiate driver
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome('chromedriver', options=options)

    #wait for page to load item, then get contents
    try:
        driver.get(baseurl.format(title))
        timeout = 5
        WebDriverWait(driver, timeout).until(ec.presence_of_element_located((By.CLASS_NAME, 'item')))
        stuff = driver.page_source
        driver.quit()

        #get the first item title and link
        soup = BeautifulSoup(stuff, features="html.parser")
        please = soup.find_all(attrs={'id':'ajaxcontent'})
        please = soup.find_all(attrs={'class':'item'})
        please = please[0].find('a')
        title = please.string
        link1 = please['href']
        
    except TimeoutException:
        link1 = None

    return title, link1

#this is the function that receives a link (supposedly a movie pate) 
#it returns any 'adaptation of' info: book title and link 
def BookSecond(link1):
    
    #initiate driver
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    driver2 = webdriver.Chrome('chromedriver', options=options)
    
    #go to the link and pull the book title and link
    try:
        driver2.get('https://www.librarything.com'+link1)
        timeout = 5
        WebDriverWait(driver2, timeout).until(ec.presence_of_element_located((By.ID, 'relationships_container')))
        stuff = driver2.page_source
        driver2.quit()

        #get the book title and link
        soup = BeautifulSoup(stuff, features="html.parser")
        yes = soup.find(attrs={'id':'relationships_container'})
        hm = soup.find(text="Is an adaptation of")
        if type(hm) == type(None):
            book = None
            link2 = None
        else:
            yes = soup.find(text="Is an adaptation of").find_next()
            yes = yes.find(attrs={'class':'popup_registered'})
            book = yes.string
            link2 = yes['href']

    except TimeoutException:
        book = None
        link2 = None

    return book, link2

#this function receives a movie title and year
#it uses the "MovieFirst" and 'BookSecond' functions
#it returns the found movie title, the link to the movie entry, the book title, and the link to the book entry
def ScrapeSearch(title, moviedate):

        #url puzzle pieces
        baseurl = 'https://www.librarything.com/search.php?search={}&searchtype=newwork_titles&searchtype=newwork_titles&sortchoice=0'
        search = title + ' ['+ str(moviedate)+ ' film]'    
   
        #search by movie and date
        newtitle, link1 = MovieFirst(baseurl, search)
        #if no results, search by movie title only
        if link1 == None:
            newtitle, link1 = MovieFirst(baseurl, title)
            #if still nothing, oh well
            if link1 == None:
                book = None
                link2 =None
            #otherwise, check that one for adapatation info  
            else:
                book, link2 = BookSecond(link1)
        #if movie results found, go into movie entry and get the book title and link
        else:
            book, link2 = BookSecond(link1)
            #if no book adaptation info, search again by just the movie title
            #assume first result without 'film]' is the book
            if link2 == None:
                book, link2 = MovieFirst(baseurl, title)
            
        #return them
        return [newtitle, link1, book, link2]
    

In [3]:
#pull in the IMDb list (1373 lines)
moviesdb = pd.read_csv('CleanMovieData.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'CleanMovieData.csv'

In [ ]:
## loop through the movie list and make a table
ohgood = pd.DataFrame(columns = ['SearchTerm', 'Link1', 'AdaptationOf', 'Link2'])

moviesdb = moviesdb.iloc[49:]

for hm in moviesdb.index:
    hello = ScrapeSearch(moviesdb.Title[hm], moviesdb.Date[hm])
    #print(hello)
    ohgood = ohgood.append(pd.Series(hello, index = ohgood.columns), ignore_index=True)

    
#NOTE: if the Search term does NOT have the [] and there is no Adaptation link, can assume the first link is the book
#NOTE: if the Search term DOES have the [], and there is no Adaptation Link, 
#maybe the first result without 'film]' is the associate book 

#none of these assumptions are coded yet we are in a hurry - should compare key words to make sure

In [356]:
print(ohgood.info())

#pulls the tag cloud - not sure where the check comes from 
#content = requests.get("http://www.librarything.com/ajax_work_makeworkCloud.php?work=3203347&check=2801929225").text
#soup = BeautifulSoup(content)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   SearchTerm    50 non-null     object
 1   Link1         50 non-null     object
 2   AdaptationOf  38 non-null     object
 3   Link2         38 non-null     object
dtypes: object(4)
memory usage: 1.7+ KB
None


In [357]:
#ohgood.to_csv('LTlinks1.csv', index = False)
#first pull was 50